In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import tensorflow as tf
from keras.utils import np_utils

import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD

In [2]:
# Helper to plot loss
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.legend()
  plt.grid(True)

# Keras, TensorFlow, and Neural Network Regression

As we have seen, neural networks aren't quite as complex as they appear at first, however we still generally don't want to have to build them from scratch very often. The libraries that we will primarily use for creating neural network models are Tensorflow and Keras. Keras and Tensorflow combine to be roughly what sklearn was for the other types of models we've used. 

### Tensorflow

Tensorflow, developed by Google, is one of the most popular libraries for neural networks. 

### Keras

Keras is another package that provides an an API offering an easier to use interface to Tensorflow, allowing us to use it with code that is higher level, avoiding much of the linear math that can make Tensorflow frustrating. Since its introduction Keras has been wrapped in with Tensorflow and the two are normally now blended together as far as we are concerned. 

### Other Alternatives

Keras and Tensorflow are not the only libraries of neural networks, the primary competitor to Tensorflow is PyTorch, which was developed by Facebook. PyTorch does pretty much the same thing as Tensorflow, we won't look at it. While PyTorch is less commonly used than Tensorflow right now, it has picked up steam recently. There's a pretty high likelihood that both Tensorflow and PyTorch will be common for the foreseeable future.

### Optimization and Efficiency Notes

Neural networks are very computationally expensive, and can take a long time to train. As we move into looking at some larger models in the near future, the processing time on a typical laptop will become prohibitive. We will utilize Google Colab for some of these examples, which will allow us to borrow some GPU processing that is often drastically faster than what we can get on our own machines.

If you happen to have a machine with a nVidia GPU, an M1/M2 Mac, or potentially a couple of other GPUs, you can install an optimized version of Tensorflow on your machine that will allow this stuff to use your GPU and be way faster without using Colab. This isn't required, but it is a good idea if your computer is capable. I don't have a GPU, so I don't have a detailed example, but you can google "install tensorflow GPU/M1 Mac" or similar for your GPU and there will be some pretty simple step-by-step instructions. If available, this will make a big difference in the speed of your models. 

## Remote Data and Google Colab

Below we'll load some data, however here we'll load something from a data repository that I setup on Azure, rather than the desktop, as we normally would. This is easy to do, we just change the path to the URL and use our "read_csv" as usual. The reason for this is that we are going to use Google Colab for some of the processing here, and if we are loading data from some static location, like this server address, we don't need to worry about loading data files and maintaining file paths. This will likely be something that you need to at least pay some attention to if you are using Colab. The key thing to remember is that it doesn't matter where the data is stored, as long as you can get to it. So if we need to change the data to make things work, we may have to change the loading bit to make it work. 

#### Your Remote Data

If you have a large-ish dataset that you're using, you can do the same thing. The datasets can be loaded directly from your github repository URL, as long as you get the "Raw" URL from Github. Click on the raw link on a file, then use the URL of the resulting page:

![Raw Link](images/github_raw.png "Raw Link")

Then you can use that URL to load the data. If you have data that comes from some site on the internet, you can likely load it similarly, assuming there's no security on the site blocking you from loading it. If there's something that doesn't work in any reasonable way, let me know, we might be able to put it on this Azure repository. I can't promise that in every situation though, it isn't free - though Azure, AWS, and Google Cloud all have free access of various levels that you could sign up for and use, if you wanted to. You can also save stuff to your Google Drive, and then load it from there - Google instructions if you need to do this. 

### Google Colab

As noted, we'll use Colab for some of our neural network stuff, as it gives us GPU access, which is massively helpful in many situations. The URL for Colab is https://colab.research.google.com/ and you can sign in with your Google account. Once on Colab, it works pretty much like Jupyter notebooks. To load data from Github, we can go to File->Open Notebook->Github, and then paste in the URL of the repository we want to load and select the notebook. Once loaded, it is basically a normal notebook. The one thing to pay attention to is that if there are any file loading bits in the code, those will fail, as noted above. 

Inside of Colab, the one key thing we want is to enable GPU accelleration. To do this, go to Edit->Notebook Settings, and then select GPU as the hardware accelerator. This will make a big difference in the speed of your models when using tensorflow. One thing to note is that while we get a bunch of GPU time for free, it is limited. So it is probably a waste to run huge tests with massive numbers of epochs, over and over, as you will likely run out of GPU time at some point. It will reset, but how much is allocated to each person is basically up to the whims of Google at any given moment. The "!" command below will print the details on the GPU that is currently being used.

I prefer to do all the editing in VS Code, save, upload to Github, then load and run the final code in a Colab window. This is more personal prefence than anything else, but it works well for me. You can also edit and save things in Colab, including back to Github, but you will likely need to do some authorization to allow access, and the process feels more annoying to me. For ease, I added this parameter to set the epochs globally for this notebook, so I can set it to a low number while working locally, then increase it when I want to test the actual performance on Colab. We could do a similar thing with sampling large datasets. 

In [3]:
BASE_EPOCHS = 100
!nvidia-smi -L

zsh:1: command not found: nvidia-smi


In [4]:
#df = pd.read_csv("data/house_data.csv")
df = pd.read_csv("https://jrssbcrsefilesnait.blob.core.windows.net/3950data1/house_data.csv")
df.tail()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
21608,360000.0,3,2.50,1530,1131,3.0,0,0,3,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,400000.0,4,2.50,2310,5813,2.0,0,0,3,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,402101.0,2,0.75,1020,1350,2.0,0,0,3,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,400000.0,3,2.50,1600,2388,2.0,0,0,3,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287
21612,325000.0,2,0.75,1020,1076,2.0,0,0,3,7,1020,0,2008,0,98144,47.5941,-122.299,1020,1357


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          21613 non-null  float64
 1   bedrooms       21613 non-null  int64  
 2   bathrooms      21613 non-null  float64
 3   sqft_living    21613 non-null  int64  
 4   sqft_lot       21613 non-null  int64  
 5   floors         21613 non-null  float64
 6   waterfront     21613 non-null  int64  
 7   view           21613 non-null  int64  
 8   condition      21613 non-null  int64  
 9   grade          21613 non-null  int64  
 10  sqft_above     21613 non-null  int64  
 11  sqft_basement  21613 non-null  int64  
 12  yr_built       21613 non-null  int64  
 13  yr_renovated   21613 non-null  int64  
 14  zipcode        21613 non-null  int64  
 15  lat            21613 non-null  float64
 16  long           21613 non-null  float64
 17  sqft_living15  21613 non-null  int64  
 18  sqft_l

In [5]:
y = np.array(df["price"]).reshape(-1,1)
X = np.array(df.drop(columns={"price"}))
print(X.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y)

(21613, 18) (21613, 1)


## Create Model

Creating a NN model is slightly different from the normal process that we are used to in sklearn. We need to do a little more work to set it up. 

### Create Model and Add Layers

First we need to make a NN model, it comes "empty". We will use a sequential model, which is the most simple type but is less configurable (which we don't care about much right now). The limitation of sequential models is that they can only take in one tensor and only output one tensor. The other options here are "functional", which allows for the structure of the model to be configured, and "model subclassing", which allows you to build almost everything from scratch. These other types are more complex and more flexible, but actually aren't really needed for most applications, and we won't use them. These more complex models are commonly used for scenarios where the data is complex, such as a self driving car - a model needs to output steering as well as velocity. Also for more complex problems such as language or image processing, this flexibility allows for models to be created that are better able to extract the information from the data.

#### Layers

Next we need to add some layers. We will start simple with only two "thinking" layers, and one to do some processing. We can think of the layers roughly like steps of the sklearn pipeline, with data entering at the first layer and predictions flowing out of the final layer. 

In addition to "normal" neural network layers, there are many other types that can do all kinds of other stuff. One example we will use here is the normalization one at the front. This layer does exactly what you'd expect - it normalizes our data so the rest of the network can use it. The normalize layer will also automatically handle the 2D nature of the data that we are used to, so we don't need to worry about that aspect here. Other layers can do everything from regularization to image processing, they are also commonly inhierited for developers to create custom layers targeting specific tasks. We'll use a few of the other ones as we move through some more complex models.

![Keras Layers](images/keras_layers.jpg "Keras Layers")

#### Dense Layers

We'll use dense layers here, and they are the main building block in our models. When adding the layer we need to specify a couple of things. One is the input dimensions - we need to tell the network what the shape of the incomming data is. 

The other argument is the units, which represents the output dimension. When using these Keras dense layers we don't need to specify each layer's input/output like we did when we made it by hand. We specify both, using units and input_dim, for the first layer that takes in the input; for subsequent layers we can just specify the output and Keras will automatically figure the rest out. 

Note that there is also an input layer that can be added, we can avoid the need for it by using the input_dim or input_shape as shown below. The two examples there do the same thing, since the input is flat - 18 features. If we are dealing with inputs that do not start out as flat, such as in an image, use the input_shape since you can specify all dimensions; we will see an example of this next time with some images. 

#### Activation Function

For each of our layers we need to define which activation function to use. For now we will use the ReLU function, which is probably the most popular. We'll look at other ones later on. Note that we've left the activation function off of the final layer - we are doing regression so we want that raw value. This is the same idea as with linear regression - we don't want the prediction to be transformed through something like the sigmoid, we just want the number.

#### Summary

After we've constructed the model, the summary command give us, well, a summary. 

We are dealing with a bunch of numerical inputs here, so we can add a normalization layer at the front end. Like with sklearn, we want to fit the normalization to the training data only. 

In [6]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(X_train))

model = Sequential()
model.add(normalizer)
model.add(Dense(18, input_shape=(18,), activation='relu'))
model.add(Dense(1))
model.summary()

2023-03-15 12:04:54.601799: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 18)               37        
 n)                                                              
                                                                 
 dense (Dense)               (None, 18)                342       
                                                                 
 dense_1 (Dense)             (None, 1)                 19        
                                                                 
Total params: 398
Trainable params: 361
Non-trainable params: 37
_________________________________________________________________


#### Compile Model

Once a model is created we need to compile it. The complie step basically builds the layers we specified above and the loss and optimization parameters below together into a usable model object. When compiling the model we are providing it with the things it needs to calculate error:

<ul>
<li> Loss - we can provide a loss function that we'd like to use. 
<li> Optimizer - the optimizer is the algorithm that the model will use to perform the gradient descent to find the lowest error. Adam is a very common choice.
<li> Learning rate - the learning rate is provided as a parameter of the optimizer. 
</ul>

##### Optimizing Adam

The optimizer is the algorithm used to perform the gradient descent and minimize error. For the most part this isn't something we need to be concerned about. The choice of optimizer is much more important if dealing with very large datasets because different optimizers have different levels of efficiency. For our purposes, we can use Adam and be pretty happy. Adam stands for Adaptive Moment Estimation which means basically that it will adjust itself depending on current gradients. It tends to be efficient both in time and memory, so it is very commonly used. 

<b>Note:</b> it is pretty common in tensorflow and keras to have things like the example below, where we can specify "adam" with either the name, or the full class with arguments. These different ways of doing things are generally interchangable, but it does make documentation more confusing, as the two different ways may often be interchanged. 

In [8]:
#model.compile(loss='mean_absolute_error', optimizer="adam")
model.compile(loss='mean_absolute_error', optimizer=tf.optimizers.Adam(learning_rate=.01))

### Fit the Model

The fit command does the same thing that we are used to, it trains the model, however there are some differences. The main difference is that batch_size is almost always set in neural networks, while the sklearn models just take all the data at once. 

What's a batch? Batches are just subsets of the data, so if the batch size is 100 the algorithm will grab 100 rows at a time before making an update to the weights and bias. There are a few reasons this exists:

<ul>
<li> Memory constraints - it is common with neural networks to deal with datasets that are extremely large. Processing data that can't fit entirely in RAM is very slow (the computer must swap data from the hard drive to RAM as it is needed) compared to data that is in RAM. Cutting the batch size can avoid this issue. 
<li> Speed - the math involved in the back propagation can sometimes be very computationally intensive. 
<li> Accuracy - batch size can have an impact on accuracy, though that impact is not very predictable. For the most part finding an optimal batch size will need to be grid-searched. 
</ul>

The fit command also has the epoch paramater, which instructs on how many times to work through ALL of the data. We want to ensure we have enough epochs to find the optimal solution. Training rounds, or epochs, are one of the key tuning factors when using neural networks. Similar to large trees, large neural networks are capable of learning the training data very well, and carry the same risk of overfitting. With neural networks, a common approach to tuning is to allow the model to train, and cut it off when we start overfitting, or when the testing accuracy starts to decrease.

#### Plot the Loss

One very common visualization we see with neural networks is a plot of both training and validation loss vs number of epochs. Generally we'll see the training loss drop - first sharply as the model initially fits itself, then more slowly as it becomes more fitted. The validation loss will usually somewhat mirror the training loss, except it will often reach a minimum at some point before again increasing. This minimum point is our best model, when the validation loss starts increasing again, that is a sign that the model has become overfitted - customized to the training data, but less and less generalizable to new data. 

Set the verbosity to 1 in the fit to get a full list of the loss for each epoch to pinpoint the exact "ideal" number of epochs. We'll look more at this in a minute. 

In [ ]:
train_log = model.fit(X_train, y_train, epochs=BASE_EPOCHS, batch_size=BASE_EPOCHS, validation_split=.2, verbose=1)
model.evaluate(X_test, y_test)
plot_loss(train_log)

### Exercise

Make a simple neural network for predicting the price of homes in California. 

In [ ]:
from sklearn.datasets import fetch_california_housing

cal = fetch_california_housing(as_frame=True)
Xcal = pd.DataFrame(cal.data)
ycal = pd.DataFrame(cal.target)
Xcal.head()

In [ ]:
ycal.head()

In [ ]:
# Model. Or whatevs. 

## Basics of Overfitting and Underfitting in Neural Networks

Just like any other type of model, our primary task in trying to attain an accurate set of predictions is to balance the overfitting and underfitting. In a neural network, the ideas are the same as with standard models, however the tools and their usage can differ slightly. 

### Add Data

Adding data to the training set is the number one way to improve accuracy. As noted above, neural networks are commonly able to acheive very high accuracy levels if provided with very large training sets. For smaller datasets, the probability of a neural network being the best model is much lower than with big data. There isn't a replacement for having large amounts of data (though there are a few tricks that we'll look at later), and modern large neural networks are (usually) the best tool that is able to take advantage of all that data. 

In the near future we'll look at some common pre-trained models that people/orgs such as Google have shared, most notably ones that do things like image recognition. These models are typically trained on really large datasets - often 10s of GB or more. This massive amount of training data allows these models to be more accurate than anything that we could create, but would be unrealistic for most people to train just due to the processing power and time needed. We can take them and adjust them a bit to our needs though...

<b>By far, the most universally beneficial way to make a neural network more accurate is to add more (good) data.</b>

### Model Capacity

The model capacity is the "size" of the model - refering to the combination of the number of neurons on each layer and the number of layers. In general the larger a feature set is, the larger a capacity we will need to be able to avoid underfitting and make accurate predictions. However, similar to a decision tree, if the model becomes too large for the data, we are likely to overfit. 

In big data scenarios (e.g. Google or Tesla training image recognition models) the feature sets can be massive (e.g. a 5 megapixel image is at least 15 million features) so the networks used have a very high capacity. Because there is a lot of training data, the model is able to have a huge capacity, but not overfit. These models can take FOREVER to process (e.g. weeks with the work paralellized on dedicated and fast machines) but they are able to make very accurate predictions since they get all the "benefits" of overfitting - predictions highly tailored to the training data; along with all the "benefits" of underfitting - since there is so much training data, they are still generalized enough to predict new data. There is something of an open question on if we should expand capacity by making the layers larger with more neurons, or by adding more layers. Like most things, the answer is whatever is tested to be best for the specific problem. In general though, more layers seems to be better for most problems, and the neural networks that excel at things such as image recognition tend to be deep learning networks - those with many layers. The combination of large datasets, deep networks, and fast processing allows for most of the modern AI that we see or interact with. 

<b>Note:</b> the size numbers that we specify as we add layers is the size of the output (other than the "input_dim" we have on the first layer). For other layers, the input of one just adapts to the output of the previous layer.

#### Capacities, For Now

We will look more at the model capacities in the next section, for now we can follow a few rough guidelines:
<ul>
<li> Input layer output size is between the # of features, and some multiple (maybe up to ~5 max) of the # of features. 
<li> Output layer is defined by the target we are predicting.  
<li> Hidden layers between the two either stay constant in size, or taper down. 
</ul>

We will look at this more next time. Unfortunately there isn't a definitive rule that always works, the ultimate decision is based on results. For now, we can just make some reasonable models and observe the results. 

In [ ]:
#Test Different Model Capacities
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(X_train))

model = Sequential()
model.add(normalizer)
model.add(Dense(128, input_dim=18, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1))
model.summary()

In [ ]:
model.compile(loss='mean_absolute_error', optimizer=tf.optimizers.Adam(learning_rate=.01))
train_log = model.fit(X_train, y_train, epochs=BASE_EPOCHS*2, batch_size=1000, validation_split=.2, verbose=0)
model.evaluate(X_test, y_test)
plot_loss(train_log)

### Early Stopping

Early stopping is very common with neural networks, due to the common pattern mentioned above of the optimal balance of over/under fitting occuring at some point within many, potentially dozens/hundreds/thousands, of epochs. Early stopping kills the process after it detects that validation loss is going back up. 

We can put early stopping in place by using a Keras function called a callback, which has odd syntax, but is quite simple to use. The patience pararmeter controls how many epcohs of worsening scores are tolerated before implementing the stop. The restore_best_weights tells the model to roll back all of its weights to the optimal point - so we automatically get the best model post-training. In most cases we probabyl want to use early stopping along with a high epoch number. We can let the model train, and just tell us when it is finished. 

Another parameter that we may want to implement is the "min_delta" parameter. This controls how much the validation loss needs to improve to be considered an improvement. If the validation loss is 0.1, and the min_delta is 0.01, then the model will stop if the validation loss goes up to 0.11, but not if it goes up to 0.12. This is useful if we have a very small validation loss, and we want to make sure that we are getting a significant improvement before stopping. In situations like the above one, where the loss is flat-ish, but bumps around a little, this may help deal with that noise and make the early stopping cutoffs work more in line with expectations.

<b>Note:</b> the patience parameter requires a little bit of thought, and a good value can vary a little between scenarios. We want to make sure that we are not stopping too early, if the patience is really small, we might get a situation where a local minima is reached and we stop the model as it is trying to escape it. If the patience is too large, we might not stop the model in time, as we might have many flat loss values, then one or two that are an improvement, resetting the clock and making the training process keep going far too long. We need to base it somewhat on what the model actually does - very stable loss results might mean we need a higher patience, while a model that is very volatile might need a lower patience, as we might not have long stable periods anywhere. Values between 5 and 15 or so are common, as are settings of around 10% of the number of epochs, but it is a good idea to test a few different values to see what works best - if you actually work with larger examples we'll run several trials with different parameters or model configurations with samples of the full dataset. These trials will likely give us an idea of a reasonable cutoff point for patience. Many of the patience settings in examples that are in the next few workbooks are probably lower than "ideal", this is entirely for practicality purposes - we might be cutting off some slightly more ideal models, but we are also saving a lot of time.

In [ ]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(X_train))

model = Sequential()
model.add(normalizer)
model.add(Dense(18, input_dim=18, activation='relu'))
model.add(Dense(18, activation='relu'))
model.add(Dense(18, activation='relu'))
model.add(Dense(1))
#model.summary()

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=50, restore_best_weights=True, min_delta=1000) 

model.compile(loss='mean_absolute_error', optimizer=tf.optimizers.Adam(learning_rate=.01))
train_log = model.fit(X_train, y_train, epochs=BASE_EPOCHS*3, batch_size=50, validation_split=.2, verbose=1, callbacks=[callback])
model.evaluate(X_test, y_test)
plot_loss(train_log)

### Validation

One change in the way we normally setup our data in neural networks is that we are performing checks during the training process with "validation data". This is another split of the data, like train-test, but this validation data is used during training to check the model's progress. This allows the model to constantly calculate the accuracy of the model on new data, similar to how things would work inside of a cross-validation in a grid search. The key thing that we can use this for is that it allows us to calculate the current "test data" performance, which is the "validation loss" that we can use to evaluate the model and cut short training. This validation dataset is normally "good enough" to be an indicator of the expected performance of the model, just like the test scores we are used to. We could still do a train-test split prior to training, and calcuate that true test accuracy, but we don't really need to. In most cases, what we referred to as test accuracy in sklearn models is just the validation accuracy in neural networks.

### Early Stopping on Validation Loss

More likely than stopping when the training loss starts to increase, we'll want to stop when the validation loss starts to increase. This is because the validation loss is a better indicator of how the model will perform on new data. This is also what allows us to set a high epoch number and walk away while training happens - as long as our model has "room to grow", or the capacity to fit the data, we can let it train until we've reached a minimum validation loss.

![Early Stopping](images/early_stopping.png "Early Stopping")

<b>Note:</b> early stopping on the validation loss will probably be the most common "optimization" that we'll do. 

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True, min_delta=2000) 

model.compile(loss='mean_absolute_error', optimizer=tf.optimizers.Adam(learning_rate=.01))
train_log = model.fit(X_train, y_train, epochs=BASE_EPOCHS*3, batch_size=50, validation_split=.3, verbose=1, callbacks=[callback])
model.evaluate(X_test, y_test)
plot_loss(train_log)

### Regularization

Like other linear models, we can implement regularization to help tame overfitting. 

We can use both L2 (Ridge) regularization that will limit growth of coefficients, and L1 (Lasso) regularization that is able to eliminate features by shrinking their coefficients to 0. The functionality is the same as we are used to, a regularization term is added to the loss, and the optimization, such as gradient descent, is then performed as normal. 

In [ ]:
# Regularization
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(X_train))

model = Sequential()
model.add(normalizer)
model.add(Dense(128, input_dim=18, activation='relu'))
model.add(Dense(128, activation='relu', kernel_regularizer="l1"))
model.add(Dense(128, activation='relu', kernel_regularizer="l2"))
model.add(Dense(1))
model.summary()

In [ ]:
model.compile(loss='mean_absolute_error', optimizer=tf.optimizers.Adam(learning_rate=.01))
train_log = model.fit(X_train, y_train, epochs=BASE_EPOCHS, batch_size=100, validation_split=.2, verbose=0)
model.evaluate(X_test, y_test)
plot_loss(train_log)

### Dropout

Neural networks also commonly employ a technique call dropouts to prevent overfitting. This works just like the name says, every time the data is moved from one layer to another some portion of the features are randomly held out from being used. The intuitive explanation for dropout is that because individual nodes in the network cannot rely on the output of the others, each node must output features that are useful on their own. This sounds somewhat weird, but is actually effective. The number of features held out is called the dropout rate, typically between .2 and .5. 

![Dropout](images/dropout.webp "Dropout")

An analogy can be drawn to the bootstrapping we looked at with trees - some random subset is selected each time, resulting in each batch getting "a slightly different look at the data", thus preventing overfitting. 

In [ ]:
# Dropout
#Test Different Model Capacities
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(X_train))

model = Sequential()
model.add(normalizer)
model.add(Dense(512, input_dim=18, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dense(1))
model.summary()

In [ ]:
model.compile(loss='mean_absolute_error', optimizer=tf.optimizers.Adam(learning_rate=.01))
train_log = model.fit(X_train, y_train, epochs=BASE_EPOCHS, batch_size=100, validation_split=.2, verbose=0)
model.evaluate(X_test, y_test)
plot_loss(train_log)

### Predictions

Once the model is trained, using it is mostly familiar to us from the sklearn stuff. 

In [ ]:
preds = model.predict(X_test)
mean_absolute_error(y_test, preds)


### Exercise

Use the California data from previously and try to add some regularization things.

In [ ]:
print(X_train_cal.shape, y_train_cal.shape)

I'll mix in a couple of regularization things, this could be almost anything. 

In [ ]:
# Dropout
#Test Different Model Capacities


### Customized Loss

Most scenarios are totally fine with a standard loss function, but what if we have something odd? What if we are playing on The Price is Right? We want to get as close as we can, without going over. We can write a loss function to mirror that! The Keras backend piece is basically a set of functions (like the "switch" we used, mean, sum, etc...) that are implemented in Tensorflow - they don't do anything differently from "normal" functions, but they are potentially more efficient.

More practically, some real life scenarios have a disperse impact of different types of error. For example, if you are working for a call centre and predicting the number of agents to staff. Having slightly too many may be an error that costs a little bit of money, but not that big of a deal. Predicting too few might incur serious penalties if callers wait and you violate an SLA. Being off in one direction is bad, being off in the other direction can cause you to "fall off of a cliff" so to speak. In cases where the impact of the error is not uniform, custom loss functions may make sense. 

<b>Note:</b> these custom loss functions work, but if you are going to put one into real use, you would likely want to look into ensuring the calculations are efficient and that you actually get a tangible benefit from the custom loss function. This one is kind of a goofy example, we will normally just want to use a standard loss function. Due to the wild differences if the residual is positive or negative, this one will likely yeild some pretty dramatic swings and variability. 

## Optional Exercise

Try to use the California data with a customized loss function. 

Note: this is a 20 way classification, so you'll probably want that many neurons on the output layer, an appropriate activation (softmax), and the y values will need to be run through np_utils.to_categorical. As well, think about the loss function, try categorical crossentropy.

We'll look at activation and loss functions more next week. 

In [ ]:
import keras.backend as K

def priceIsRight(y_true, y_pred):
    return K.switch(
                    y_pred <= y_true, 
                    (y_true - y_pred) ** 2,
                    (y_true - y_pred) ** 4
                    )

In [ ]:
# Use Custom Function 

## Big Exercise - Newsgroup Classification

Try to classify the newsgroup data. 

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

remove = ("headers", "footers", "quotes")

data_train = fetch_20newsgroups(
    subset="train", shuffle=True, remove=remove)

data_test = fetch_20newsgroups(
    subset="test", shuffle=True, remove=remove)

In [ ]:
news_tf = TfidfVectorizer(sublinear_tf=True, ngram_range=(1,3), stop_words="english", strip_accents="unicode")
X_train = news_tf.fit_transform(data_train.data)
y_train = data_train.target
X_test = news_tf.transform(data_test.data)
y_test = data_test.target
print("Train:", X_train.shape, "  Test:", X_test.shape)

In [ ]:
# Modelling

## Neural Network Tuning Summary vs Reality

The techniques above generally expose a pattern that we can use to make accurate models:
<ul>
<li> Create a model that is accurate and overfitted. (Or setup process to generate an overfitted model)
<li> Use hyperparameter tuning-ish methods to trial several different models. 
<li> Use tools such as regularization and early stopping to "trim" the overfitting back. 
</ul>

For the most part, for what we are doing, this will probably work fine, without forcing us to put a tonne of thought into things upfront; we can just make a large model, then reduce it. In practice the main downside to this type of brute force approach is time, and by extension, cost. For us the datasets are mostly small enough that in an extreme case we could do something like setup a bunch of trials, let our computer train and test overnight, and wake up to a model that is pretty good. If our data was scaled up by a factor of 10,000 this becomes less practical. We would want to sample the data to make each trial run much more quickly, but we'd still be dealing with a non-trivial amount of processing time for each model that we want to try. Doing things that reduce the processing requirements of each epoch, such as feature selection, choosing a "correctly" sized model, or smart sets of hyperparameters to try, will reduce the amount of "bad" trials, and allow us to dedicate more time to "good" trials. This is one of the, relatively rare, scenarios that paying attention to processing time can have massive impacts on the end results. We'll look more at making a good sized model next. 